In [84]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

In [85]:
df = pd.read_csv('diabetes.csv',sep = ',')
df['DiabetesPedigreeFunction'] = df['DiabetesPedigreeFunction']*100
df.DiabetesPedigreeFunction = df.DiabetesPedigreeFunction.astype(int)
df.BMI = df.BMI.astype(int) 
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33,62,50,1
1,1,85,66,29,0,26,35,31,0
2,8,183,64,0,0,23,67,32,1
3,1,89,66,23,94,28,16,21,0
4,0,137,40,35,168,43,228,33,1


In [88]:
dataset = df.to_numpy()
X_train_num = dataset[:500,0:8]
X_test_num = dataset[500:,0:8]
y_train_num = dataset[:500,8]
y_test_num = dataset[500:,8]

In [89]:
X_train_den = np.full(X_train_num.shape, 1) 
X_test_den = np.full(X_test_num.shape, 1) 
y_train_den = np.full(len(y_train_num),1)
y_test_den = np.full(len(y_test_num),1)

In [90]:
X_train_den[:,6] = X_train_den[:,6]*100

In [91]:
X_train_den

array([[  1,   1,   1, ...,   1, 100,   1],
       [  1,   1,   1, ...,   1, 100,   1],
       [  1,   1,   1, ...,   1, 100,   1],
       ...,
       [  1,   1,   1, ...,   1, 100,   1],
       [  1,   1,   1, ...,   1, 100,   1],
       [  1,   1,   1, ...,   1, 100,   1]])

In [92]:
#shortening numners 
def shorter(c,d):
    c = int(c)
    d = int(d)
    if(c<0):
        n = c*(-1)
        if(len(str(n))==len(str(d)) and len(str(n))>3):      
            a = f"{n:.0f}"
            a = str(a[:-3])
            a = int(a)*(-1)
            b = f"{d:.0f}"
            b = str(b[:-3])
            b = int(b)
        elif(len(str(n))>=3 and len(str(d))>=3):
            a = f"{n:.0f}"
            a = str(a)
            b = f"{d:.0f}"
            b = str(b)
            if(len(str(a))<len(str(b))):
                s = len(str(a))-2
            else:
                s = len(str(b))-2
            a = int(a[:-s])
            b = int(b[:-s])
            a = a*-1 
        else:
            a = int(n*-1)
            b = d
    elif(len(str(c))==len(str(d)) and len(str(c))>3):      
            a = f"{c:.0f}"
            a = str(a[:-2])
            a = int(a)
            b = f"{d:.0f}"
            b = str(b[:-2])
            b = int(b)
#     elif(len(str(c))==1 and len(str(d))>3):
#         a = int(0)
#         b = int(1)
    elif(len(str(c))>3 and len(str(d))>3):
            a = f"{c:.0f}"
            a = str(a)
            b = f"{d:.0f}"
            b = str(b)
            if(len(str(a))<len(str(b))):
                s = len(str(a))-3
            else:
                s = len(str(b))-3
            a = int(a[:-s])
            b = int(b[:-s])
    else:
        a = c
        b = d
    return a,b

In [93]:
#condensed function multiply and add matrix with numerator and denominator one row at a time
# Python Program to find the L.C.M. of two input number
from math import gcd
def compute_lcm(x, y):
   # choose the greater number 
    if (x > y):
        greater = x
    else:
        greater = y
    while(True):
        if(x==0):
            lcm = y
            break
        elif(y==0):
            lcm = x
            break
        elif((greater % x == 0) and (greater % y == 0)):
            lcm = greater
            break
        greater += 1
    return lcm
 
#Subtracting 2 numbers of the form a/b -c/d
def subtraction(a,b,c,d):
    den = compute_lcm(b,d)
    if b ==0:
        num = c ; den = d
    elif d==0:
        num= a ; den = b
    else:    
        num = ((den/b)*a) - ((den/d)*c)
    return num,den

#adding array of elements of the form x/y
def add_array(a_num,a_den):
    #will work for an int array of any length
    lcm = 1;sum1 =0
    for i in a_den:
        lcm = int(lcm*i/gcd(lcm, i))
#     print("lcm is ",lcm)
    for i in range(len(a_num)):
        sum1 = sum1 + (a_num[i]*(a_den[i]/lcm))
    return sum1,lcm

#adding two numbers of the form a/b + c/d
def addition(a,b,c,d):
    den = compute_lcm(b,d)
    if b ==0:
        num = c ; den = d
    elif d==0:
        num= a ; den = b
    else:    
        num = ((den/b)*a) + ((den/d)*c)
    return int(num),int(den)

#multiplying two arrays of fraction and adding them  for row by column multiplication
def multiply(a_num,a_den,b_num, b_den):
#     print(type(a_num))
    z_num = np.zeros(len(a_num)).astype(int)
    z_den = np.zeros(len(a_den)).astype(int)
#     print("printing anum,aden,bnum,bden",a_num,a_den,b_num,b_den)                                   
    for i in range(len(a_num)):
        z_num[i]=(a_num[i]*b_num[i])     
        z_den[i]=(a_den[i]*b_den[i])
    
    x = z_num.tolist()
    y = z_den.tolist()
#     print("Printing z_num and z_den", x , y )
    num,den = add_array(x,y)
#     print("multiplication results is",num,den) 
    return num,den
    

In [94]:
#claculate y_hat =wx+b | x is in the form a single array and w is in the form of array too
# need toc hange w into wnum and wden
def y_hat(w_num,w_den,x_n,x_d,bnum, bden):
    temp_n, temp_d = multiply(x_n,x_d,w_num,w_den)
#         print("printing temporary values",temp_n,temp_d)
    y_hat_num,y_hat_den = addition( temp_n, temp_d,bnum,bden)
    y_hat_num,y_hat_den = shorter(y_hat_num,y_hat_den)   #trying this
#     print("y_hat values for this iterationsa are",y_hat_num,y_hat_den)
    return y_hat_num,y_hat_den

In [133]:
# taylor series works when x<1
# Assumption based on values of x
# need to check the bwlow formulas of taylor series one more time
# x and y are numerator and denominator
def sigmoidfrac(x,y):
    if(x/y>=10):
        num = 1; den = 1
    elif((x/y)>=9 and (x/y)<10):
        num = 99988; den = 100000
    elif((x/y)>=8 and (x/y)<9):
        num = 99966; den = 100000
    elif((x/y)>=7 and (x/y)<8):
        num = 99909; den = 100000
    elif((x/y)>=6 and (x/y)<7):
        num = 99753; den = 100000
    elif((x/y)>=5 and (x/y)<6):
        num = 99331; den = 100000
    elif((x/y)>=1 and (x/y)<5):
        q = int(x/y) 
        r = x%y
        num= pow(19,q)*24*pow(y,4)
        den= pow(19,q)*24*pow(y,4)+pow(7,q)*(24*pow(y,4)-24*r*pow(y,3)+12*pow(r,2)*pow(y,2)-4*y*pow(r,3)+pow(r,4))    
    elif(x/y>-1 and x/y<1):
        num= 24*pow(y,4)
        den= 48*pow(y,4)-24*x*pow(y,3)+12*pow(x,2)*pow(y,2)-4*y*pow(x,3)+pow(x,4)   
    elif(x/y>-6 and x/y<=-1):
        q = int(x/y) 
        r = x%y
        num = 24*pow(y,4)*pow(7,-q)
        den = 24*pow(y,4)*pow(7,-q)+pow(19,-q)*(24*pow(y,4)+24*r*pow(y,3)+12*pow(r,2)*pow(y,2)+4*y*pow(r,3)+pow(r,4))
    elif(x/y>-7 and x/y<=-6):
        num =15; den= 10000 
    elif(x/y>-8 and x/y<=-7):
        num =6; den = 10000     
    elif(x/y>-9 and x/y<=-8):
        num =2; den = 10000 
    elif(x/y>-10 and x/y<=-9):
         num = 7; den = 100000
    elif(x/y<=-10):
         num = 0 ; den =1
    return num,den

In [117]:
#calculating mean square values
def mse(y_hat_num,y_hat_den,y_num,y_den):
    num=0;den =0
    for i in range(len(y_hat_num)):
        num1,den1=subtraction(y_num[i],y_den[i],y_hat_num[i],y_hat_den[i])
        num1 = num1*num1
        den1 = den1*den1
        num,den = addition(num,den,num1,den1)
    den = den * len(y_den)
#     print(num,den)
    return num,den

In [156]:
#calculating subtraction of Y_hat Y
# getting error 
def y_hat_substraction(y_hat_num,y_hat_den,y_num,y_den):
    y_diffn =np.zeros(y_hat_num.shape[0])
    y_diffd =np.zeros(y_hat_num.shape[0])
    for i in range(len(y_hat_num)):
        y_diffn[i],y_diffd[i] = subtraction(y_hat_num[i],y_hat_den[i], y_num[i], y_den[i])
    return y_diffn,y_diffd



In [119]:
# get sigmoid derivative
# define derivative of numerator and denominator
def sigmoid_derivative(x,y):
    num = x*y-pow(x,2)
    den = pow(y,2)
    return num,den

In [172]:
#multiply error and sigmoid derivative
# this is useful for calcualting delta b and updating bias
def delta_b(sign, sigd, errorn, errord):
    delta_bn = np.zeros(len(sign))
    delta_bd = np.zeros(len(sign))
    for i in range(len(delta_bn)):
        delta_bn[i] = sign[i]* errorn[i]
        delta_bd[i] = sigd[i]* errord[i]
    return delta_bn, delta_bd

In [178]:
# multiply error and sigmoid and input, 
# this will give us a delta w for calculating new weights
def delta_w(inputn, inputd, delbn,delbd):
    inputn = inputn.T
    inputd = inputd.T
    delw_n = np.zeros(inputn.shape[0])
    delw_d = np.zeros(inputd.shape[0])
    temp_n = np.zeros(inputn.shape[1])
    temp_d = np.zeros(inputd.shape[1])
    print(inputn.shape)
    print(delbn.shape)
    for i in range(inputn.shape[0]):
        delw_n[i], delw_d[i] = multiply(inputn[i], inputd[i], delbn, delbd)
    return delw_n, delw_d

In [180]:
# get new weights and biases
def get_new_weights(lrn, lrd, dw_num,dw_den,w_num,w_den): 
    for i in range(len(dw_num)):
        dw_num[i]= dw_num[i]*lrn
        dw_den[i]= dw_den[i]*lrd
    for i in range(len(dw_num)):
        w_num[i],w_den[i] = subtraction(w_num[i],w_den[i],dw_num[i],dw_den[i]) 
    return w_num, w_den
    

In [193]:
def get_new_bias(lrn, lrd, delb_num, delb_den,bnum, bden ):
   
    for i in range(len(sigdern)):
        bt_num = lrn * delb_num[i]
        bt_den = lrd * delb_den[i] 
        bnum,bden = subtraction(bnum,bden,bt_num,bt_den,)
    return bnum,bden 

In [124]:
#normalize weights and bias
def normalize_weights_bias(wnum,wden, bnum,bden):
    wn_num = np.zeros(10) 
    wn_den = np.zeros(10)
    for i in range (len(wf_num)):
        wn_num[i] = int((wnum[i]/wden[i]) * 100)
        wn_den[i] = 100
    bnum = int(bnum/bden * 100)
    bden = 100
    
#     print("Normalizing weights and biases",wn_num,wn_den, bnum, bden)
    return wn_num, wn_den,bnum, bden

In [125]:
wnum = np.ones(10)
wden = np.ones(10)
bnum =1
bden =1
lr = [1,100]

In [126]:
X_train_num[0]


array([  6, 148,  72,  35,   0,  33,  62,  50], dtype=int64)

In [144]:
# get yhat
yh_n =np.zeros(len(y_train_num))
yh_d =np.zeros(len(y_train_num))
for i in range (X_train_num.shape[0]):
    xn = X_train_num[i]
    xd = X_train_den[i]
    yh_n[i],yh_d[i]= y_hat(wnum,wden,xn,xd,bnum,bden)
# print(yh_n, yh_d)

In [145]:
# get sigmoid 
sign = np.zeros(len(yh_n))
sigd = np.zeros(len(yh_d))
for i in range(len(yh_n)):
    sign[i], sigd[i] = sigmoidfrac(yh_n[i], yh_d[i])
    sign[i] = int(sign[i]/sigd[i]*1000)
    sigd[i] = 1000
# print(sign,sigd)

In [146]:
# get mean square error
msen,msed = mse(sign, sigd, y_train_num, y_train_den)
print(msen,msed)

213575416 500000000


In [157]:
# get subtractino of sigmoid and labels
ydifn = np.zeros(len(y_train_num))
ydifd = np.zeros(len(y_train_num))
ydifn, ydifd = y_hat_substraction(sign,sigd,y_train_num,y_train_den)

In [159]:
# print(ydifn, ydifd)

In [161]:
# get sigmoid derivative
sigdern = np.zeros(len(ydifn))
sigderd = np.zeros(len(ydifd))
for i in range(len(sigdern)):
    sigdern[i],sigderd[i] = sigmoid_derivative(sign[i],sigd[i])



In [173]:
# multiply error and sig derivative
delta_bn, delta_bd = delta_b(sign, sigd, ydifn, ydifd)  

In [174]:
print(len(delta_bn))

500


In [179]:
# get delta w by multiplying sig derivative with inputs
delta_wn, delta_wd = delta_w(X_train_num, X_train_den, delta_bn,delta_bd)

(8, 500)
(500,)


In [183]:
lrn =1
lrd =100

In [209]:
wnum, wden = get_new_weights(lrn,lrd,delta_wn,delta_wd,wnum,wden)

In [210]:
print(wnum, wden)

[-5.05443093e+10 -1.94937467e+12 -1.24235690e+12 -3.63638155e+11
 -1.23224253e+12 -5.25261786e+11 -8.41778298e+09 -5.55564675e+11
  1.00000000e+02  1.00000000e+02] [1.e+12 1.e+12 1.e+12 1.e+12 1.e+12 1.e+12 1.e+14 1.e+12 1.e+02 1.e+02]


In [204]:
bn, bd = get_new_bias(lrn,lrd, delta_bn, delta_bd,bnum,bden)

In [205]:
print(bn,bd)

-81608416.0 100000000.0


In [206]:
for i in range(len(wnum)):
    wnum[i]= int(wnum[i]/wden[i]*100)
    wden[i]= 100

In [207]:
print(wnum,wden)

[  -5. -193. -123.  -36. -122.  -52.    0.  -55.  100.  100.] [100. 100. 100. 100. 100. 100. 100. 100. 100. 100.]
